In [5]:
%pip install openpyxl

import requests
import pandas as pd


In [ ]:
# Load Excel file
file_path = 'orte_geonames.xlsx'
df_excel = pd.read_excel(file_path)


# Output all rows to the console
# Set display options to show all rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Optional: Rename column for clarity
df_excel.rename(columns={'Unnamed: 1': 'geoname_id'}, inplace=True)

# Create new DataFrame with 'geoname_id' entries
data = {
    'geoname_id': df_excel['geoname_id'].tolist()
}
df = pd.DataFrame(data)

# GeoNames credentials
username = 'kzfry97'

# Function to fetch coordinates
def fetch_coordinates(geoname_id):
    url = 'http://api.geonames.org/getJSON'
    params = {
        'geonameId': geoname_id,
        'username': username
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        if 'lat' in data and 'lng' in data:
            return data['lat'], data['lng']
        else:
            print(f"Coordinates not found for ID {geoname_id}")
            return None, None
    except requests.RequestException as e:
        print(f"Request failed for ID {geoname_id}: {e}")
        return None, None

# Add latitude and longitude columns
df['latitude'] = None
df['longitude'] = None

# Populate coordinates
for index, row in df.iterrows():
    lat, lng = fetch_coordinates(row['geoname_id'])
    df.at[index, 'latitude'] = lat
    df.at[index, 'longitude'] = lng

# Show final DataFrame
print(df)

# Save to Excel
df.to_excel('geonames_table.xlsx', index=False)

KeyError: 'geoname_id'